In [1]:
import pymysql
import numpy as np

db = pymysql.connect(host = "cdb-r2g8flnu.bj.tencentcdb.com", port = 10209, user = "dase2020", password = "dase2020", database = "dase_intro_2020")
cursor = db.cursor()  # 使用 cursor() 方法创建一个游标对象 cursor,执行SQL语句都是通过游标对象实现

# sql = "SELECT VERSION();"  # 该SQL语句返回MySQL的安装版本，用以确定是否成功连接服务器

# cursor.execute(sql)  # 执行SQL语句
# result = cursor.fetchone()  # 获取单条数据
# print(result)

In [2]:
# 从第18条记录开始的5条记录
sql = "SELECT * FROM bicycle_train LIMIT 17, 5;"
cursor.execute(sql)
result = cursor.fetchall() #获取单条数据
print('编号 城市 小时 工作日 大气温度 体感温度 天气 风速 单车租借量')
# 自动循环，行 -> 列
for record in result:
    for element in record:
        print("%3d" %element, end = ' ')
    print() #一组数据换行

编号 城市 小时 工作日 大气温度 体感温度 天气 风速 单车租借量
 18   1  17   1   8   4   1   2  75 
 19   1   1   1  18  19   1   3   5 
 20   1   7   0  23  25   2   1  13 
 21   0   9   0  25  27   1   0  67 
 22   0  16   1  11  11   3   2  76 


In [3]:
# 数据表中wind取值范围
sql = "SELECT DISTINCT wind FROM bicycle_train ORDER BY wind;"
cursor.execute(sql)
result = cursor.fetchall() # 默认返回二维tuple
print("风速最小值为%d 风速最大值为%d" % (result[0][0], result[-1][0]))

风速最小值为0 风速最大值为7


In [4]:
# 满足城市为北京市，10时，晴天，无风或1级风，租用单车数量不小于100条件下大气温度的平均值
sql = "SELECT AVG(temp_air) FROM bicycle_train WHERE city = 0 AND hour = 10 AND weather = 1 AND wind BETWEEN 0 AND 1 AND y >= 100;"
cursor.execute(sql)
# result = cursor.fetchall()
# print("%.1f" %result[0])
result = cursor.fetchone() #只获取一行
print("%.1f" %result)

20.6


In [5]:
# 满足城市为北京市，10时，晴天，无风或1级风，租用单车数量不小于100条件下大气温度的方差
sql = "SELECT temp_air FROM bicycle_train WHERE city = 0 AND hour = 10 AND weather = 1 AND wind BETWEEN 0 AND 1 AND y >= 100;"
cursor.execute(sql)
result = cursor.fetchall()
air = []
for item in result:
    air.append(item[0])
air = np.array(air)
mean_air = np.mean(air)
var_air = (sum((air - mean_air) ** 2)) / len(air)
print(var_air)

37.64


In [6]:
# 分城市显示工作日雨雪天单车租用总量，并降序排序
# 默认升序排序
sql = "SELECT city, SUM(y) FROM bicycle_train WHERE is_workday = 1 AND weather = 3 GROUP BY city ORDER BY SUM(y) DESC;"
cursor.execute(sql)
result = cursor.fetchall()
city_dict = {0: '北京', 1: '上海'} # 方便输出城市名称
for num in result:
    print("%s:%d" % (city_dict[num[0]], num[1]))

上海:9106
北京:8084


In [7]:
# 分别查询17时至19时每小时上海市在工作日且体感温度不大于10摄氏度时租用单车的平均值（四舍五入至整数），并且结果按升序排序
sql = "SELECT hour, AVG(y) FROM bicycle_train WHERE is_workday = 1 AND temp_body <= 10 AND city = 1 AND hour BETWEEN 17 AND 19 GROUP BY hour ORDER BY AVG(y);"
cursor.execute(sql)
result = cursor.fetchall()
for item in result:
    print("%d时：%d辆" %(item[0], round(item[1])))

19时：42辆
18时：63辆
17时：65辆
